## Wrapper: test randomized smoothing

In [ ]:
%load_ext autoreload
%autoreload 2

import torch 
from torch import nn

import torchvision
import torchvision.transforms as transforms

import numpy as np
from matplotlib import pyplot as plt

import random_smooth as rs
from random_smooth import RandSmoothedClassifier
from base_classifier import ConvBase

# correct device handling?
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
base_classifier = ConvBase().to(device)
number_classes = 10
sigma = 1
batch_size = 128
epochs = 1

In [ ]:
# --------------------

### Load and prepare data
(see https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False, num_workers=2)
# choose batch_size 1 for certification

In [ ]:
# --------------------

### Train the smoothed classifier
(see https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)

In [ ]:
# init the smoothed classifier
classifier_smoothed = RandSmoothedClassifier(base_classifier=base_classifier, c=number_classes, sigma=sigma).to(device)

In [ ]:
# choose a loss function and optimizer
loss_func = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(classifier_smoothed.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(classifier_smoothed.parameters(), lr=0.001)

In [ ]:
# set model to training mode
classifier_smoothed.train()
# base_classifier.train()
losses = []
accuracies = []
for epoch in range(epochs):
    print(f'Starting epoch {epoch}')
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + compute accuracy
        outputs = classifier_smoothed(inputs)
        # outputs = base_classifier(inputs)
        _, predicted = torch.max(outputs, 1)
        accuracy = torch.mean((predicted == labels) * 1.0).item()
        accuracies.append(accuracy)
        
        # compute loss + backward
        loss = loss_func(outputs, labels)
        loss.backward()
        losses.append(loss.item())
        
        # optimization
        optimizer.step()

print('Finished training')

In [ ]:
PATH = './cifar_net.pth'
torch.save(classifier_smoothed.state_dict(), PATH)

In [ ]:
plt.plot(losses)
plt.title("Loss development over iterations") 
plt.xlabel("Iteration") 
plt.ylabel("Loss") 
plt.grid(True)
plt.show()

In [ ]:
plt.plot(accuracies)
plt.title("Accuracy development over iterations") 
plt.xlabel("Iteration") 
plt.ylabel("Accuracy") 
plt.grid(True)
plt.show()

In [ ]:
# --------------------

### Evaluate smoothed classifier
(see https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)

In [ ]:
number_samples = 100
number_samples0 = 1000
alpha = 0.1
batch_size = batch_size

In [ ]:
# load model
model = RandSmoothedClassifier(base_classifier=base_classifier, c=number_classes, sigma=sigma).to(device)
model.load_state_dict(torch.load(PATH))

In [ ]:
correct_robust_radii = []
false_robust_radii = []
abstain = 0

for (image, label) in test_loader:
    prediction, robust_radius = model.certify(image, number_samples, number_samples0, alpha, batch_size)
    
    if prediction > -1:
        if prediction == label:
            correct_robust_radii.append(robust_radius)
        else:
            false_robust_radii.append(robust_radius)
    else:
        abstain += 1
        
correct = len(correct_robust_radii)
false = len(false_robust_radii)

In [ ]:
# average robust radius
if len(correct_robust_radii) > 0:
    torch.mean(correct_robust_radii)